In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Load the data
df = pd.read_csv("matched_sensor_data.csv")
df = df[["temperature", "humidity", "soundLevel", "event"]]

# Manual mapping: fire → 1, normal → 0
df["event_encoded"] = df["event"].map({"normal": 0, "fire": 1})

# Features and target
X = df[["temperature", "humidity", "soundLevel"]].values
y = df["event_encoded"].values

# Train/test split (stratify to keep class ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# Create a Normalization layer and adapt to training data
normalizer = layers.Normalization(input_shape=(3,))
normalizer.adapt(X_train)

In [ ]:
# Build the model
model = models.Sequential([
    normalizer,  # scales inputs automatically
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

# Compile
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16, 
    validation_split=0.1,
    verbose=1
)

In [ ]:
# Evaluate
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy:.4f}")

# Predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Classification report
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
labels = ["normal", "fire"]

# Plot Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Save model in TensorFlow Serving format
tf_serving_path = "models/fire_nn/1"
os.makedirs(tf_serving_path, exist_ok=True)

# Export in SavedModel format for TensorFlow Serving
model.export(tf_serving_path)